In [1]:
import pandas as pd
import sqlalchemy as sql
import numpy as np
# 

In [2]:
xls = pd.ExcelFile('./database.xlsx')
df1 = pd.read_excel(xls, '工业产品')
df2 = pd.read_excel(xls, '能源产品')
df3 = pd.read_excel(xls, '生活产品')
df3 = df3.drop(df3.columns[[15, 16, 17]], axis=1)
df4 = pd.read_excel(xls, '废弃物处理')
df5 = pd.read_excel(xls, '交通服务')
df6 = pd.read_excel(xls, '碳汇')

df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, df6.shape


((379, 15), (173, 15), (362, 15), (61, 15), (45, 15), (67, 15))

In [3]:
# df1.shape
# df1.info()

result = pd.concat([df1[1:], df2[1:], df3[1:], df4[1:], df5[1:], df6[1:]], ignore_index=True, sort=False)
result.shape


(1081, 15)

In [4]:
result = result.fillna(0)
# result.head(1)

In [5]:
# result['unit'] = np.where( result['上游排放单位\n（Unit）'].isnull() , result['上游排放单位\n（Unit）'], result['下游排放单位\n（Unit）'])

In [6]:
sum_factor = []
delete = []
for idx, row in result.iterrows():
    # print(row.values[3], row.values[5])
    value = row.values[3] + row.values[5]
    sum_factor.append(value)  
    delete.append(row.values[4] != 0 and row.values[6] != 0 and row.values[4] != row.values[6])
    
result['sum_factor'] = sum_factor
result['delete'] = delete


In [7]:
# result = result[result['delete'] == False]
duplicate = result[result['delete'] == True]
for idx, row in duplicate.iterrows():
  print(idx)
duplicate



386
390
482
483
484
801
802


,1级分类\n（Level 1）,2级分类\n（Level 2）,3级分类\n（Level 3）,上游排放\n（Upstream emissions）,上游排放单位\n（Unit）,下游排放\n（Downstream emissions）,下游排放单位\n（Unit）,排放环节\n（Emission processes）,排放温室气体占比\n（GHG percentage）,数据时间\n（Year）,不确定性\n（Uncertainty）,其他\n（Others）,参考文献/数据来源\n（Data source）,负责人\n（Principal）,参考文献/简化\n（Data source）,sum_factor,delete
386,煤炭及煤制品\n（Coal and coal products）,煤制品（煤制油除外）\n（Coal products）,焦炉气\n（Coke Oven Gas）,0.32,吨二氧化碳当量/吨\n(t CO2-eq/ t),0.86,千克二氧化碳当量/立方米\n（kg CO2-eq/Nm3),生产：0.323；燃烧：0.66,下游：CO2:1.0,上游：2006-2008；下游：2011-2015,（1）全国/典型个案；（2）理论计算/实测和计算结合；（3）结果定量不确定性（下游：1.71...,表中数据为来源文献中调研的十家企业对应数据的平均值；焦炉煤气的密度为0.4~0.5kg/Nm3,"[1] 侯萍,王洪涛,张浩,范辞冬,黄娜.用于组织和产品碳足迹的中国电力温室气体排放因子 [...",孟凡鑫/樊兆宇/张建军/王柯,"侯萍等, 2012;\n白宗庆等, 2016;\n国家发改委, 2014",1.18,True
390,煤炭及煤制品\n（Coal and coal products）,煤制品（煤制油除外）\n（Coal products）,发生炉煤气\n（By gas furnace）,0.30,千克二氧化碳当量/吨\n（kg CO2-eq/t）,0.90,千克二氧化碳当量/吨\n(kgCO2-eq/ t),生产+输送：0.3 kgCO2-eq/ t；终端消费：0.9kgCO2-eq/ t,CO2：1.0,2015-2019,（1）全国/典型个案；（2）理论计算/实测和计算结合,表中数据为来源文献中综合考虑冷、热煤气得到的数值,"苑卫军,王辉,韩明汝,杨征.发生炉煤气生命周期碳排放研究 [J]. 洁净煤技术, 2019,...",孟凡鑫/樊兆宇/张建军/王柯,"苑卫军等, 2019",1.20,True
482,热能\n( Thermal energy）,生物质能供热\n（ Biomass heating）,生物质能供热-平均\n（ Biomass heating）,39.66,千克二氧化碳当量/吨生物质\n（kgCO2-eq/t）,149.54,千克二氧化碳当量/吨\n（kgCO2-eq/t）,0,0,0,0,0,"[1] 冯新新.秸秆成型燃料集中供暖温室气体减排研究 [D].北京:中国农业科学院, 202...",杜鸣溪,"冯新新, 2021;\n耿爱欣等, 2020",189.20,True
483,热能\n( Thermal energy）,生物质能供热\n（ Biomass heating）,秸秆\n (Straw),46.40,千克二氧化碳当量/吨生物质\n（kgCO2-eq/t）,28.00,千克二氧化碳当量/吨\n（kgCO2-eq/t）,收储运:7.71；原料加工:25.72；供热:12.96,上游:CO2:1；下游:CH4:0.638；N2O:0.362,2021,（1）区域（甘肃省临洮县）；（2）实测和计算结合,0,"冯新新.秸秆成型燃料集中供暖温室气体减排研究 [D].北京:中国农业科学院, 2021.",杜鸣溪,"冯新新, 2021",74.40,True
484,热能\n( Thermal energy）,生物质能供热\n（ Biomass heating）,薪柴\n (Wood),32.91,千克二氧化碳当量/吨生物质\n（kgCO2-eq/t）,271.08,千克二氧化碳当量/吨\n（kgCO2-eq/t）,0,0,0,（1）全国；（2）理论计算；（3）结果定量不确定性（下游：10.169 -531.999 ）,0,"耿爱欣,潘文琦,杨红强.中国林木生物质能源替代煤炭的减排效益评估 [J]. 资源科学, 20...",杜鸣溪,"耿爱欣等, 2020",303.99,True
801,电器\n（Household appliances）,空调\n（air conditioning）,R290家用空调\n（R290 household air conditioning）,89.16,千克二氧化碳当量/台\n（kgCO2-eq/台）,0.43,千克二氧化碳当量/台,原料获取：78；制造：8.38；运输：2.78；使用中泄露：0.43kg，使用中电力消耗：2...,只有CO2,2019,（1）个案；（2）实测和计算结合,文献中的电力排放因子是0.592kgCO2/kwh，已折算为2020年电网排放因子,"李小燕,宁前,何国庚.采用R290和R32的家用空调器全生命周期碳排放研究 [J]. 低温工...",方雪坤，胡孝依,"李小燕等, 2021",89.59,True
802,电器\n（Household appliances）,空调\n（air conditioning）,R32家用空调\n（R32 household air conditioning）,86.04,千克二氧化碳当量/台\n（kgCO2-eq/台）,157.95,千克二氧化碳当量/台,原料获取：74.8；制造：8.38；运输：2.86；使用中泄露：157.95kg，使用中电力...,只有CO2,2019,（1）个案；（3）实测和计算结合,文献中的电力排放因子是0.592kgCO2/kwh，已折算为2020年电网排放因子,"李小燕,宁前,何国庚.采用R290和R32的家用空调器全生命周期碳排放研究 [J]. 低温工...",方雪坤，胡孝依,"李小燕等, 2021",243.99,True


In [8]:
result.at[801, "下游排放单位\n（Unit）"] = result.at[801, '上游排放单位\n（Unit）']
result.at[802, "下游排放单位\n（Unit）"] = result.at[802, '上游排放单位\n（Unit）']
result.at[309, "下游排放单位\n（Unit）"] = result.at[309, '上游排放单位\n（Unit）']
result.at[482, "下游排放单位\n（Unit）"] = result.at[482, '上游排放单位\n（Unit）']
result.at[483, "下游排放单位\n（Unit）"] = result.at[483, '上游排放单位\n（Unit）']
result.at[484, "下游排放单位\n（Unit）"] = result.at[484, '上游排放单位\n（Unit）']
result.at[390, "下游排放单位\n（Unit）"] = result.at[390, '上游排放单位\n（Unit）']

result.head()

,1级分类\n（Level 1）,2级分类\n（Level 2）,3级分类\n（Level 3）,上游排放\n（Upstream emissions）,上游排放单位\n（Unit）,下游排放\n（Downstream emissions）,下游排放单位\n（Unit）,排放环节\n（Emission processes）,排放温室气体占比\n（GHG percentage）,数据时间\n（Year）,不确定性\n（Uncertainty）,其他\n（Others）,参考文献/数据来源\n（Data source）,负责人\n（Principal）,参考文献/简化\n（Data source）,sum_factor,delete
0,金属\n（Metal）,黑色金属矿\n（Ferrous metal ore）,铁矿石平均\n（Iron ore average）,1.10,吨二氧化碳当量/吨\n（t CO2-eq/ t）,0.0,0,铁矿石开采和选矿：1.10,0,2014,0,0,"[1] 冯超,朱海玲,徐志强.基于EIO LCA的铁矿石采选温室气体排放强度分析 [J]. ...",0,"冯超等, 2017;\n刘宏强等, 2016;\n万腾方等, 2018",1.10,False
1,金属\n（Metal）,黑色金属矿\n（Ferrous metal ore）,铁矿石（Iron ore）,2.14,吨二氧化碳当量/吨\n（t CO2-eq/ t）,0.0,0,铁矿石开采和选矿：2.138；,CH4：0.195；N2O：0.011；CO2：1.932,2014,（1）区域（全国）,铁矿石开采和选矿，CH4（34）；N2O（298）；,"[1] 冯超, 朱海玲, 徐志强.基于EIO LCA的铁矿石采选温室气体排放强度分析 [J]...",张琦,"冯超等, 2017;\nWorld Steel Association,2021",2.14,False
2,金属\n（Metal）,黑色金属矿\n（Ferrous metal ore）,铁矿石（Iron ore）,0.07,吨二氧化碳当量/吨\n（t CO2-eq/ t）,0.0,0,铁矿石开采和选矿：0.07,0,2014,0,根据文献中折算（开采1.34t铁矿石，排放0.094CO2当量）,"[1] 刘宏强,付建勋,刘思雨,谢欣悦,杨笑楹.钢铁生产过程二氧化碳排放计算方法与实践 [J...",张琦,"刘宏强等, 2016;\n万腾方等, 2018",0.07,False
3,金属\n（Metal）,黑色金属矿\n（Ferrous metal ore）,锰矿石（Manganese ore）,0.12,吨二氧化碳当量/吨\n（t CO2-eq/ t）,0.0,0,矿石开采：0.063；流通阶段：0.052；深加工：0.824,0,2003-2015,（1）区域（贵州）；（2）调研，计算；,数据来源于对研究区（贵州铜仁松桃）内的 3 家锰矿山、8 家电解锰企业和相关科研机构调研,"刘江宜,窦世权,肖建忠,张洁,潘文.基于全生命周期的矿产资源开发环境影响评估——以贵州松桃锰...",孟凡鑫，樊兆宇,"刘江宜等, 2019",0.12,False
4,金属\n（Metal）,有色金属矿\n（Nonferrous metal ore）,铝土矿平均（Bauxite average）,0.04,吨二氧化碳当量/吨\n（t CO2-eq/ t）,0.0,0,0,0,0,0,0,"[1] Cai B F, Liang S, Zhou J, et al. China Hig...",蔡博峰,"Cai B F et al, 2018;\n李贵奇, 2011",0.04,False


In [9]:
# result.loc[len(result.index)] = result.iloc[[386]]

result = result.append(result.loc[[386]])
# result.at[len(result.index) -1 , 0 ] =  len(result.index)
# result.tail()


C:\Users\markd\AppData\Local\Temp\ipykernel_8324\1819149932.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(result.loc[[386]])


In [10]:
first = []
second = []

unit = []
english_unit = []
for idx, row in result.iterrows():
  if idx == 386:
    print(1)

  # initial check
  if row.values[4] != 0 and row.values[6] != 0 and row.values[4] != row.values[6] :
    print(idx)
  if row.values[4] == 0 and row.values[6] == 0:
    print('hh')


  value = 0
  if row.values[4] == 0:
    value = row.values[6]
  else:
    value = row.values[4]

  # final check
  if value == 0:
    print(1)
  unit.append(value)



  c = "（"
  if "（" not in value:
    c = "("
  id_start = value.index(c)
  id_end = value.index("CO")
  target = value[id_start+1 : id_end]
  target = target.strip()
  first.append(target)



  c_second = "）"
  if "）" not in value:
    c_second = ")"
  new_value = value[id_start:]
  second_start = new_value.index("/")
  second_end = new_value.index(c_second)
  r = value[id_start+second_start+1 : id_start+second_end]
  if r == "件" or r == "部" or r == "台" or r == "个" or r == "本":
    r = 'unit'
  if r == " (person·km)":
    r = "person·km"
  if r == "( t·km)":
    r = "t·km"
  r = r.strip()
  if r == "Nm³":
    r = "Nm3"
  if r == 'kwh' or r == 'kWh':
    r = 'KWh'
  

  second.append(r)


result['final_unit'] = unit
result['first'] = first
result['second'] = second
result.drop('delete', axis=1)

result.to_excel('output.xlsx')


1
386
1
386
